In [1]:
from mbmvpa.preprocessing.events import LatentProcessGenerator

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


In [2]:
root = "/data2/project_model_based_fmri/piva_dd"
save_path = "/data2/project_modelbasedMVPA/piva_dd_mbmvpa_zoomed_2"
mask_path = "/data2/project_modelbasedMVPA/ds000005/derivatives/fmriprep/masks"

In [3]:
dm_model = "dd_hyperbolic"

def example_adjust(row):
    ## rename data in a row to the name which can match hbayesdm.dd_hyperbolic requirements ##
    if row["delay_left"] >= row["delay_right"]:
        row["delay_later"] = row["delay_left"]
        row["delay_sooner"] = row["delay_right"]
        row["amount_later"] = row["money_left"]
        row["amount_sooner"] = row["money_right"]
        row["choice"] = 1 if row["choice"] == 1 else 0
    else:
        row["delay_later"] = row["delay_right"]
        row["delay_sooner"] = row["delay_left"]
        row["amount_later"] = row["money_right"]
        row["amount_sooner"] = row["money_left"]
        row["choice"] = 1 if row["choice"] == 2 else 0
    return row

def example_filter(row):
    # in the paper, the condition for trial varies in a single run,
    # agent == 0 for making a choice for him or herself
    # agent == 1 for making a choice for other
    # to consider only non-social choice behavior, select only the cases with agent == 0
    return row["agent"] == 0


def example_latent(row, param_dict):
    # calculate subjective utility for choosing later option over sooner option
    # hyperbolic discount function is adopted
    ev_later = row["amount_later"] / (1 + param_dict["k"] * row["delay_later"])
    ev_sooner  = row["amount_sooner"] / (1 + param_dict["k"] * row["delay_sooner"])
    modulation = ev_later - ev_sooner
    row["modulation"] = modulation
    return row


In [5]:
y_generator = LatentProcessGenerator(bids_layout=root,
                                     bids_controller=None,
                                     save_path=save_path,
                                     task_name=None,
                                     process_name="subjectiveutility",
                                     adjust_function=example_adjust,
                                     filter_function=example_filter,
                                     latent_function=example_latent,
                                     modulation_dfwise=None,
                                     dm_model=dm_model,
                                     filter_for_modeling=None,
                                     individual_params=None,
                                     hrf_model="glover",
                                     use_duration=False,
                                     n_core=4,
                                     ignore_original=False)


INFO: start loading BIDSLayout


/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


INFO: fMRIPrep is loaded
INFO: task name is not designated. find most common task name
INFO: selected task_name is SIC
INFO: MB-MVPA is loaded
INFO: 158 file(s) in Original & fMRIPrep.
INFO: individual parameters are loaded


In [6]:
y_generator.run(overwrite=True)

1it [00:00,  5.43it/s]

INFO: indivudal parameters table
    subjID         k      beta
0        1  0.007130  1.075816
1        2  0.035188  0.098425
2        3  0.004005  0.715110
3        4  0.009414  1.270404
4        5  0.002058  1.316499
5        6  0.000778  0.765669
6        7  0.004329  0.978210
7        8  0.006049  0.819824
8        9  0.051175  0.975056
9       10  0.002009  0.997599
10      11  0.002397  1.754058
11      12  0.001642  1.471127
12      13  0.033747  0.111113
13      14  0.000290  0.602368
14      15  0.000929  0.941105
15      16  0.000826  1.816373
16      17  0.000217  1.224327
17      18  0.008286  0.623351
18      19  0.004227  1.406870
19      20  0.015498  1.205835
INFO: start processing 158 events.[task SIC, process subjectiveutility]


158it [00:08, 18.00it/s]

INFO: events processing is done.
